In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os 
import glob
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
mother_path = 'drive/MyDrive/Find/Train'
folders=os.listdir(mother_path)
x = []
y = []
for i in folders:
  if 'Jay' in i:
    label = 0 # spill X
  elif 'Alex' in i :
    label = 1 # spill
  elif 'Derek' in i:
    label=2
  else:
    label=3
  files=glob.glob(os.path.join(mother_path,i)+'/*')
  for i,file in enumerate(files):
    if i % 20 == 0:
      print(i,'/',len(files))
    im = Image.open(file)
    im=im.resize((200,200))
    im = im.convert('RGB')
    data = np.asarray(im)
    x.append(data)
    y.append(label)
x = np.array(x)

y = np.array(y)

x = x.astype('float32')/ 255
y = y.astype('int')
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.1)

#######preprocessing###############

x_train.shape
x_test.shape
  

0 / 113
20 / 113
40 / 113
60 / 113
80 / 113
100 / 113
0 / 149
20 / 149
40 / 149
60 / 149
80 / 149
100 / 149
120 / 149
140 / 149
0 / 50
20 / 50
40 / 50


(32, 200, 200, 3)

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
model = tf.keras.applications.InceptionV3(weights='imagenet',include_top=False,input_shape=(200,200,3))
x = model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
x =  tf.keras.layers.Dropout(0.2)(x)
y =  tf.keras.layers.Dense(1,activation='sigmoid')(x)
history_1 = model_final = Model(inputs = model.input, outputs = y)
model_final.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
history_1 = model_final.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
9/9 [==============================] - 125s 13s/step - loss: 0.0000e+00 - accuracy: 0.5429 - val_loss: 0.0000e+00 - val_accuracy: 0.4062
Epoch 2/10
5/9 [===============>..............] - ETA: 49s - loss: 0.0000e+00 - accuracy: 0.8062 

In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [26]:
train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Find/Train',
        target_size=(200, 200),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/Find/Test',
        target_size=(200, 200),
        batch_size=32,
        class_mode='categorical')
history = model_final.fit(
        train_generator,
        steps_per_epoch=10,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=10)

Found 312 images belonging to 4 classes.
Found 0 images belonging to 4 classes.
Epoch 1/10


InvalidArgumentError: ignored